<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/riva_asr_asr-python-advanced-customize-vocabulary-and-lexicon/nvidia_logo.png" style="width: 90px; float: right;">

<a id='isc-task-description'></a>
# Language Modeling

### Task Description

Language modeling returns a probability distribution over a sequence of words. Besides assigning a probability to a sequence of words, the language models also assign a probability for the likelihood of a given word (or a sequence of words) that follows a sequence of words. <br>

> The sentence:  **all of a sudden I notice three guys standing on the sidewalk**
> would be scored higher than 
> the sentence: **on guys all I of notice sidewalk three a sudden standing the** by the language model. <br>

A language model trained on large corpus can significantly improve the accuracy of an ASR system as suggested in recent research.

### N-gram Language Model
There are primarily two types of language models:

- **N-gram language models**: These models use frequency of n-grams to learn the probability distribution over words. Two benefits of N-gram language models are simplicity and scalability – with a larger `n`, a model can store more context with a well-understood space–time tradeoff, enabling small experiments to scale up efficiently.
- **Neural language models**: These models use different kinds of neural networks to model the probability distribution over words, and have surpassed the N-gram language models in the ability to model language, but are generally slower to evaluate.

In this tutorial, we will show how to train, evaluate, and optionally fine-tune an [N-gram language model](https://web.stanford.edu/~jurafsky/slp3/3.pdf) leveraging TAO Toolkit.

Prerequisite: You have access and are logged into NVIDIA NGC. For step-by-step instructions, refer to the [NGC Getting Started Guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#registering-activating-ngc-account).

---
## Let's Dig in: Training and Finetuning LM with KenLM and NeMo

### Installing and setting up NeMo
Cloning and installing NeMo from source.

In [ ]:
## Install NeMo
BRANCH = 'main'
!git clone https://github.com/NVIDIA/NeMo.git
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

### Installing and setting up KenLM
Installing KenLM from source

In [ ]:
!apt install libeigen3-dev
!git clone https://github.com/kpu/kenlm.git
!cd kenlm && mkdir build && cd build && cmake .. && make -j

!pip3 install git+https://github.com/kpu/kenlm.git
!pip3 install git+https://github.com/flashlight/text.git

### Installing and setting up NGC CLI
To install and setup the NGC CLI, follow the instrutions from [here](https://ngc.nvidia.com/setup/installers/cli)

---
<a id='isc-prepare-data'></a>
### Preparing the dataset
#### Librispeech LM Normalized dataset
For this tutorial, we use the normalized version of the LibriSpeech LM dataset to train our N-gram language model. The normalized version of the LibriSpeech LM dataset is available [here](https://www.openslr.org/11/).<br>
The training data is publicly available [here](https://www.openslr.org/resources/11/librispeech-lm-corpus.tgz) and can be downloaded directly.
#### Downloading the dataset

In [15]:
# Set the path to a folder where you want your data and results to be saved.
DATA_DOWNLOAD_DIR="content/datasets"
RESULTS_DIR="content/results"
MODELS_DIR="content/models"

!mkdir -p $DATA_DOWNLOAD_DIR $RESULTS_DIR $MODELS_DIR

In [ ]:
# NOTE: Ensure that wget and unzip utilities are available. If not, install them.
!wget 'https://www.openslr.org/resources/11/librispeech-lm-norm.txt.gz' -P $DATA_DOWNLOAD_DIR

# Extract the data
!gzip -dk $DATA_DOWNLOAD_DIR/librispeech-lm-norm.txt.gz

For the sake of reducing the time this demo takes, we reduce the number of lines of the training dataset. Feel free to modify the number of used lines.

In [6]:
# Use a random 100,000 lines for training
!shuf -n 100000 $DATA_DOWNLOAD_DIR/librispeech-lm-norm.txt  > $DATA_DOWNLOAD_DIR/reduced_training.txt

## Download Evaluation dataset


In [ ]:
#Note: This data can be used only with NVIDIA’s products or services for evaluation and benchmarking purposes.
!source ~/.bash_profile && ngc registry resource  download-version --dest $DATA_DOWNLOAD_DIR nvstaging/tao/healthcare_eval_dataset:1.0

## Generating Base Language Model

In [8]:
KENLM_BASE="kenlm/build/bin/"

`$KENLM_BASE/lmplz`<br>
Required Arguments:

*   `-o`: Order of the language model to estimate.

Optional Arguments:

*   `-S`: Memory to use. This is a number followed by single-character suffix: % for percentage of physical memory (on platforms where this is measured), b for bytes, K for kilobytes, M for megabytes, and so on for G and T. If no suffix is given, kilobytes are assumed for compatability with GNU sort. The sort program is not used; the command line is simply designed to be compatible.
*   `-T`: Temporary file location.
*   `--discount_fallback`: Kneser-Ney smoothing discounts are estimated from counts of counts, including the number of singletons.


In [ ]:
!$KENLM_BASE/lmplz -o 4 < $DATA_DOWNLOAD_DIR/reduced_training.txt > $RESULTS_DIR/base_lm.arpa

`$KENLM_BASE/build_binary`<br>
Arguments:

*   `-q`: Quantization flag for the probabilities. (For eg. `-q 8` store 8 bits of probability)
*   `-b`: Quantization flag for backoff weights. (For eg. `-b 7` store 7 bits of backoff)
*   `-a`: Maximum number of bits to be removed and stored implicitly using a table of offsets to reduce memory footprint.



In [ ]:
!$KENLM_BASE/build_binary  trie -q 8 -b 7 -a 256 $RESULTS_DIR/base_lm.arpa $RESULTS_DIR/base_lm.bin

## Load ASR model

In [ ]:
import torch
from nemo.collections.asr.models import ASRModel
from nemo.collections.asr.metrics.wer import CTCDecodingConfig

### Download English Conformer model from NGC

In [ ]:
!source ~/.bash_profile && ngc registry model download-version "nvidia/riva/speechtotext_en_us_conformer:trainable_v3.1" --dest $MODELS_DIR

### Create Lexicon file for flashlight decoder

In [ ]:
!cd NeMo/scripts/asr_language_modeling/ngram_lm/ && python create_lexicon_from_arpa.py --arpa $RESULTS_DIR/base_lm.arpa --model $MODELS_DIR/speechtotext_en_us_conformer_vtrainable_v3.1/Conformer-CTC-L_spe-1024_en-US_Riva-ASR-SET-3.1.nemo --lower --dst $RESULTS_DIR

### Updating decoder type, Language model and lexicon

In [ ]:
device = torch.device('cuda')
asr_model = ASRModel.restore_from(f"{MODELS_DIR}/speechtotext_en_us_conformer_vtrainable_v3.1/Conformer-CTC-L_spe-1024_en-US_Riva-ASR-SET-3.1.nemo").to(device)


decoding_cfg = CTCDecodingConfig()

decoding_cfg.strategy = "flashlight"
decoding_cfg.beam.search_type = "flashlight"
decoding_cfg.beam.kenlm_path = f'{RESULTS_DIR}/base_lm.bin'
decoding_cfg.beam.flashlight_cfg.lexicon_path=f'{RESULTS_DIR}/base_lm.lexicon'
decoding_cfg.beam.beam_size = 32
decoding_cfg.beam.beam_alpha = 0.2
decoding_cfg.beam.beam_beta = 0.2
decoding_cfg.beam.flashlight_cfg.beam_size_token = 32
decoding_cfg.beam.flashlight_cfg.beam_threshold = 25.0

asr_model.change_decoding_strategy(decoding_cfg)

### Evaluate

In [19]:
import json
import os

def transcribe_json(asr_model, json_path, output_json):
    dataset_root = os.path.split(json_path)[0]
    with open(json_path) as fin, open(output_json, 'w') as fout:
        manifest = []
        audios = []
        for line in fin:
            dt = json.loads(line.strip())
            manifest.append(dt)
            audios.append(dt['audio_filepath'].replace("/data", dataset_root))
        transcripts = asr_model.transcribe(paths2audio_files=audios)
        for i in range(len(transcripts)):
            dt = {
                'audio_filepath': manifest[i]['audio_filepath'],
                'text': transcripts[i]
            }
            fout.write(json.dumps(dt)+"\n")



In [ ]:
transcribe_json(asr_model, f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/general.json", f"{RESULTS_DIR}/general_base_lm.json")
transcribe_json(asr_model, f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/healthcare.json", f"{RESULTS_DIR}/healthcare_base_lm.json")

### Calculate word error rate


In [ ]:
!pip install jiwer
from jiwer import wer
import json

def calculate_wer(ground_truth_manifest, asr_transcript):
    data ={}
    ground_truths = []
    predictions = []
    with open(ground_truth_manifest) as file:
        for line in file:
            dt = json.loads(line)
            data[dt['audio_filepath']] = dt['text']
    with open(asr_transcript) as file:
        for line in file:
            dt = json.loads(line)
            if dt['audio_filepath'] in data:
                ground_truths.append(data[dt['audio_filepath']])
                predictions.append(dt['text'])
    return round(100*wer(ground_truths, predictions), 2)



In [ ]:
print( "WER of base model on generic domain data", calculate_wer(f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/general.json", f"{RESULTS_DIR}/general_base_lm.json"))

In [ ]:
print("WER of base model on Healthcare domain data", calculate_wer(f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/healthcare.json", f"{RESULTS_DIR}/healthcare_base_lm.json"))

---
<a id='isc-Finetuning'></a>
## Finetuning/Interpolation


The fine-tuning process will continue training using a previously trained model by training a second model on new domain data and interpolating it with the original model. Finetuning requires the original model have intermediate enabled during training. A finetuned model cannot be used for finetuning again. <br>


### Downloading and procesing domain data (healthcare) for LM finetuning
For the purpose of finetuning on healthcare domain we can make use of Kaggle dataset PubMed 200k RCT: a Dataset for Sequential Sentence Classification in Medical Abstracts. [https://arxiv.org/abs/1710.06071] <br>
This dataset is available at https://www.kaggle.com/datasets/anshulmehtakaggl/200000-abstracts-for-seq-sentence-classification <br>
Please follow the instructions to install and authenticate Kaggle API https://www.kaggle.com/docs/api 
<br>
**Note**: *Each user is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use*


In [ ]:
!kaggle datasets download -d anshulmehtakaggl/200000-abstracts-for-seq-sentence-classification

In [ ]:
!unzip -d $DATA_DOWNLOAD_DIR 200000-abstracts-for-seq-sentence-classification.zip

In [34]:
# Perform basic text cleaning and generate domain data
import string,re
def clean_text(text):
    text = re.sub(r"[^a-z' ]+", "", text.lower().strip())
    text = ' '.join(text.split())
    if len(text.split())> 5:
        return text.strip()
    
# Using dev file since we want a small amount of finetuning data. For better text Normalization use NeMo [https://github.com/NVIDIA/NeMo/tree/main/nemo_text_processing]
with open(f'{DATA_DOWNLOAD_DIR}/20k_abstracts_numbers_with_@/dev.txt') as file, open(f'{DATA_DOWNLOAD_DIR}/domain_data_all.txt', 'w') as outfile:
    for line in file:
        if line.startswith("###") or not line.strip():
            continue
        _, text = line.strip().split('\t')
        text = clean_text(text)
        if text:
            outfile.write(text+'\n')
            
# Picking top 10000 lines from dataset
!head -10000 $DATA_DOWNLOAD_DIR/domain_data_all.txt > $DATA_DOWNLOAD_DIR/domain_data.txt

The fine-tuning process will continue training using a previously trained model by training a second model on new domain data and interpolating it with the original model. Finetuning requires the original model have intermediate enabled during training. A finetuned model cannot be used for finetuning again. <br>


For Finetuning a N-gram language model with KenLM, follow the following steps:


1.   Generate intermediate arpa files.
2.   Interpolate


### Generating internediate arpa

In [ ]:
# Base LM
!mkdir base_intermediate
!$KENLM_BASE/lmplz -o 4 --intermediate base_intermediate/inter < $DATA_DOWNLOAD_DIR/reduced_training.txt

# Healthcare LM
!mkdir healthcare_intermediate
!$KENLM_BASE/lmplz -o 4 --intermediate healthcare_intermediate/inter < $DATA_DOWNLOAD_DIR/domain_data.txt

#### Interpolate
Weights for interpolation can be passed with
```
 -w 0.6 0.4
```
Here, 60% weightage is assigned to base LM and 40% to Domain

In [ ]:
!$KENLM_BASE/interpolate -w 0.6 0.4 -m base_intermediate/inter healthcare_intermediate/inter > $RESULTS_DIR/interpolated_lm_60-40.arpa
!$KENLM_BASE/build_binary  trie -q 8 -b 7 -a 256 $RESULTS_DIR/interpolated_lm_60-40.arpa $RESULTS_DIR/interpolated_lm_60-40.bin

In [ ]:
!cd NeMo/scripts/asr_language_modeling/ngram_lm/ && python create_lexicon_from_arpa.py --arpa $RESULTS_DIR/interpolated_lm_60-40.arpa --model $MODELS_DIR/speechtotext_en_us_conformer_vtrainable_v3.1/Conformer-CTC-L_spe-1024_en-US_Riva-ASR-SET-3.1.nemo --lower --dst $RESULTS_DIR

#### Evaluate

In [ ]:
decoding_cfg.beam.kenlm_path = f'{RESULTS_DIR}/interpolated_lm_60-40.bin'
decoding_cfg.beam.flashlight_cfg.lexicon_path=f'{RESULTS_DIR}/interpolated_lm_60-40.lexicon'

asr_model.change_decoding_strategy(decoding_cfg)

In [ ]:
transcribe_json(asr_model, f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/general.json", f"{RESULTS_DIR}/general_interpolated_lm_60-40.json")
transcribe_json(asr_model, f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/healthcare.json", f"{RESULTS_DIR}/healthcare_interpolated_lm_60-40.json")

#### Calculate WER

In [ ]:
print( "WER of base model on generic domain data", calculate_wer(f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/general.json", f"{RESULTS_DIR}/general_base_lm.json"))
print( "WER of Domain model on generic domain data", calculate_wer(f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/general.json", f"{RESULTS_DIR}/general_interpolated_lm_60-40.json"))

In [ ]:
print("WER of base model on Healthcare domain data", calculate_wer(f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/healthcare.json", f"{RESULTS_DIR}/healthcare_base_lm.json"))
print("WER of Domain model on Healthcare domain data", calculate_wer(f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/healthcare.json", f"{RESULTS_DIR}/healthcare_interpolated_lm_60-40.json"))

### Pruning

LM generated by simply passing the text corpus to TAO toolkit contains some n-grams which are less frequent(in corpus) and thus have very low probabilities. Such n-grams can be removed by `pruning`.<br>
Pruning requires some thresholds which can be passed with `–prune` parameter followed with space separated thresholds specifying count thresholds for each order while generating ARPA:
```
 –prune 0 1 7 9
```

All the n-gram with frequncy less than or equal to specified threshold will get eliminated.<br>
Here, 2-grams with freq. <= 1, 3-gram with freq.<=7 & 4-gram with freq.<=9 will get eliminated.<br>
There's a tradeoff between degree of pruning and accuracy. High pruning parameters will reduce the size of language model but at the cost of model accuracy!  

#### *Note:
Pruning of 1-gram is not supported, threshold for 1-gram should always be 0

In [ ]:
!kenlm/build/bin/lmplz -o 4 --prune 0 1 7 9  < $DATA_DOWNLOAD_DIR/reduced_training.txt > $RESULTS_DIR/pruned_lm.arpa

In [ ]:
!$KENLM_BASE/build_binary  trie -q 8 -b 7 -a 256 $RESULTS_DIR/pruned_lm.arpa $RESULTS_DIR/pruned_lm.bin

In [ ]:
!cd NeMo/scripts/asr_language_modeling/ngram_lm/ && python create_lexicon_from_arpa.py --arpa $RESULTS_DIR/pruned_lm.arpa --model $MODELS_DIR/speechtotext_en_us_conformer_vtrainable_v3.1/Conformer-CTC-L_spe-1024_en-US_Riva-ASR-SET-3.1.nemo --lower --dst $RESULTS_DIR

In [ ]:
# Lets check the size of original LM and Pruned LM
!echo "Size of unpruned ARPA: $(du -h $RESULTS_DIR/base_lm.arpa | cut -f 1)"
!echo "Size of pruned ARPA: $(du -h $RESULTS_DIR/pruned_lm.arpa | cut -f 1)"

#### Evaluate

In [ ]:
decoding_cfg.beam.kenlm_path = f'{RESULTS_DIR}/pruned_lm.bin'
decoding_cfg.beam.flashlight_cfg.lexicon_path=f'{RESULTS_DIR}/base_lm.lexicon'

asr_model.change_decoding_strategy(decoding_cfg)

In [ ]:
transcribe_json(asr_model, f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/general.json", f"{RESULTS_DIR}/general_pruned_lm.json")
transcribe_json(asr_model, f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/healthcare.json", f"{RESULTS_DIR}/healthcare_pruned_lm.json")

#### Calculate WER

In [ ]:
print( "WER of base model on generic domain data", calculate_wer(f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/general.json", f"{RESULTS_DIR}/general_base_lm.json"))
print( "WER of Pruned base model on generic domain data", calculate_wer(f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/general.json", f"{RESULTS_DIR}/general_pruned_lm.json"))

In [ ]:
print("WER of base model on Healthcare domain data", calculate_wer(f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/healthcare.json", f"{RESULTS_DIR}/healthcare_base_lm.json"))
print("WER of Pruned base model on Healthcare domain data", calculate_wer(f"{DATA_DOWNLOAD_DIR}/healthcare_eval_dataset_v1.0/healthcare.json", f"{RESULTS_DIR}/healthcare_pruned_lm.json"))